In [1]:
# Import the required libraries and dependencies
import numpy as np
import pandas as pd

In [2]:
# Read the data from the apple-price.csv file into a Pandas DataFrame
df_stock = pd.read_csv(
    "../Resources/aapl-price.csv", 
    index_col="date", 
    parse_dates=True, 
    infer_datetime_format=True
)

# Review the DataFrame
df_stock.head()

/var/folders/cf/y6vhhqvs245bxntt_6f300v00000gp/T/ipykernel_72922/597171049.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_stock = pd.read_csv(


,close
date,
2015-10-19,27.93
2015-10-20,28.44
2015-10-21,28.44
2015-10-22,28.88
2015-10-23,29.77


In [18]:
# Read the data from the apple-trends.csv file into a Pandas DataFrame
df_trends = pd.read_csv(
    "../Resources/apple-trends.csv", 
    index_col="date", 
    parse_dates=True, 
    infer_datetime_format=True
)

# Review the DataFrame
df_trends.head()

/var/folders/cf/y6vhhqvs245bxntt_6f300v00000gp/T/ipykernel_72922/98708399.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_trends = pd.read_csv(


,trend-worldwide
date,
2015-10-23,32
2015-10-30,33
2015-11-06,32
2015-11-13,32
2015-11-20,31


#### Step 2: Concatenate the `df_stock` DataFrame to the `df_trends` DataFrame, creating a single DataFrame named `df_apple`.

> **Rewind** Concatenation: Two DataFrames that have the same `datetime` index can be merged using `pd.concat`. The `axis=1` parameter means that we're bringing the columns of the two DataFrames together.

In [19]:
# Concatenate Apple's stock price and Google trends data
# Use a parameter of axis=1 to ensure the concatenation by columns
# Chain the dropna function to remove any rows of data that do not 
# contain information in both columns
df_apple = pd.concat([df_stock, df_trends], axis=1).dropna()



# Review the df_apple DataFrame
df_apple.head()


,close,trend-worldwide
date,,
2015-10-23,29.77,32.0
2015-10-30,29.88,33.0
2015-11-06,30.27,32.0
2015-11-13,28.09,32.0
2015-11-20,29.83,31.0


### Step 3: Create a plot of the concatenated DataFrame and analyze any trends and correlations. Do you observe any seasonal patterns?

> **Hint** You might find it useful to refer to the [Apple Events page](https://www.apple.com/apple-events/) and the [Timeline of Apple Inc. products](https://en.wikipedia.org/wiki/Timeline_of_Apple_Inc._products) to spotlight moments over time that could have triggered a particular pattern.

In [20]:
# Use hvplot to visualize the time series data in the df_apple DataFrame
import hvplot.pandas
df_apple.hvplot()


/Users/juanlavierigoiri/anaconda3/lib/python3.11/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/Users/juanlavierigoiri/anaconda3/lib/python3.11/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:NdOverlay   [Variable]
   :Curve   [date]   (value)

## Building Time Trends

### Step 1: Note that on September 10, 2019, Apple organized an event where it presented the new iPhone 11 family, the Apple Watch Series 5, and a new iPad. This event gained worldwide attention.

### Step 2: Closely examine the data from March 1, 2019 to January 31, 2020.

In [21]:
# Using the df_apple DataFrame, use the loc function to select a
# range of data from March 1st, 2019 to January 31st, 2020
df_apple_2019_2020 = df_apple.loc["2019-03-01":"2020-01-31"]



# Review the df_spotlight DataFrame
df_apple_2019_2020.head()


,close,trend-worldwide
date,,
2019-03-01,43.74,27.0
2019-03-08,43.23,27.0
2019-03-15,46.53,27.0
2019-03-22,47.76,30.0
2019-03-29,47.49,34.0


### Step 3: Create a plot and identify whether both time series indicate a common trend that might correspond to this narrative.

In [22]:
# Visualize the df_spotlight DataFrame
df_apple_2019_2020.hvplot()



/Users/juanlavierigoiri/anaconda3/lib/python3.11/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/Users/juanlavierigoiri/anaconda3/lib/python3.11/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:NdOverlay   [Variable]
   :Curve   [date]   (value)

### Step 4: Before seeking any correlations between these time series, add columns to the `df_apple` DataFrame to analyze the impact of the Google Trends data on the weekly returns and stock volatility, as follows:

1. Use the Pandas `shift` function to add a new column that lags the Google Trends data by one period.

    > **Hint** Google Trends data is reported every week on Sunday, so you have a weekly period in this time series.

2. Use the Pandas `pct_change` function to add a new column that has the weekly price returns.

3. Compute the rolling stock volatility for Apple by using the following Pandas method:

    `df_apple["weekly_volatility"] = df_apple["close"].pct_change().rolling(window=4).std()`


In [28]:
# Create a column which lags Google trends by one week
# Use the shift function, and move the data down by one row
df_apple["lagged_trends"] = df_apple["trend-worldwide"].shift(1)



In [29]:
# Create a column that contains the Apple weekly return data
# Use the pct_change function to calculate the weekly return values
df_apple["weekly_return"] = df_apple["close"].pct_change()




In [30]:
# Create a column of Apple weekly rolling stock volatility 
# Chain the pct_function, the rolling function and a 4 period window, and the std function 
# to calculate the standard deviation of the weekly return data
df_apple["weekly_volatility"] = df_apple["weekly_return"].rolling(window=4).std() * np.sqrt(52)



In [31]:
# Display the df_apple DataFrame to confirm all columns are present
df_apple.head()


,close,trend-worldwide,weekly_volatility,weekly_return,lagged_trends
date,,,,,
2015-10-23,29.77,32.0,NaN,NaN,NaN
2015-10-30,29.88,33.0,NaN,0.003695,32.0
2015-11-06,30.27,32.0,NaN,0.013052,33.0
2015-11-13,28.09,32.0,NaN,-0.072019,32.0
2015-11-20,29.83,31.0,0.399258,0.061944,32.0


### Step 5: Use the Pandas `corr` function to compute the correlations among the lagged Google Trends data, the price returns, and the stock volatility. Does any predictable relationship exist?

In [32]:
# Use the corr function to compute the correlation between the lagged Google Trends data, price returns, and stock volatility
df_apple.corr()



,close,trend-worldwide,weekly_volatility,weekly_return,lagged_trends
close,1.000000,0.249017,0.197668,0.157472,0.249998
trend-worldwide,0.249017,1.000000,-0.014795,0.019526,0.735369
weekly_volatility,0.197668,-0.014795,1.000000,0.086676,0.004911
weekly_return,0.157472,0.019526,0.086676,1.000000,0.026701
lagged_trends,0.249998,0.735369,0.004911,0.026701,1.000000
